# Testing Individual components of the FV HE scheme

In [1]:
import random

from syft.frameworks.torch.he.fv.modulus import CoeffModulus
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.key_generator import KeyGenerator
from syft.frameworks.torch.he.fv.encryptor import Encryptor
from syft.frameworks.torch.he.fv.decryptor import Decryptor
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.modulus import SeqLevelType
from syft.frameworks.torch.he.fv.evaluator import Evaluator

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'



## Keygeneration

In [2]:
poly_modulus = 8
bit_sizes= [10,10]
plain_modulus = 8
ctx = Context(EncryptionParams(poly_modulus, CoeffModulus().create(poly_modulus, bit_sizes), plain_modulus))
keygenerator = KeyGenerator(ctx)
sk, pk = keygenerator.keygen()

In [3]:
print(ctx.param.coeff_modulus)

[977, 1009]


In [4]:
# print(len(sk.data))
print('secret key values : ', sk.data)

secret key values :  [[1, 1, 976, 0, 1, 0, 0, 976], [1, 1, 1008, 0, 1, 0, 0, 1008]]


In [5]:
# print(pk.data)
# print('public key values : ', pk.data)

## Integer Encoder
Encodes Integer values to Plaintext object

In [6]:
int_encoder = IntegerEncoder(ctx)
ri1 = random.randint(0,100)
ri2 = random.randint(0,100)
ri3 = random.randint(0,100)
pt1 = int_encoder.encode(ri1)
pt2 = int_encoder.encode(ri2)
pt3 = int_encoder.encode(ri3)
print(pt1.data,"   ", pt2.data, "   ", pt3.data)
# print('plaintext data',plaintext.data)

[1, 1, 1, 1, 0, 0, 1]     [0, 1, 0, 0, 1, 1]     [0, 1]


### Decodes back to Integer

In [7]:
print(int_encoder.decode(pt1))
print(int_encoder.decode(pt2))
print(int_encoder.decode(pt3))

79
50
2


## Encrypter
Encrypt Plaintext to ciphertext using public_key

In [8]:
encrypter = Encryptor(ctx, pk)

In [9]:
ct1 = encrypter.encrypt(pt1)
ct2 = encrypter.encrypt(pt2)
ct3 = encrypter.encrypt(pt3)

Encrypt Plaintext to ciphertext using secret_key

## Decryptor
Decrypt Ciphertext to Plaintext using secret_key

In [10]:
decrypter = Decryptor(ctx, sk)

In [11]:
dec1 = decrypter.decrypt(ct1)
dec2 = decrypter.decrypt(ct2)
dec3 = decrypter.decrypt(ct3)

[[1, 1, 976, 0, 1, 0, 0, 976], [1, 1, 1008, 0, 1, 0, 0, 1008]]



sk_power :  [[[1, 1, 976, 0, 1, 0, 0, 976], [1, 1, 1008, 0, 1, 0, 0, 1008]]]



phase [c0 + c1 * sk + c2 * sk^2 ...]_q. :  [[122, 112, 126, 124, 9, 975, 126, 7], [126, 116, 130, 128, 9, 1007, 130, 7]]



temp_product_modq :  [[122, 112, 126, 124, 9, 975, 126, 7], [126, 116, 130, 128, 9, 1007, 130, 7]]



result "Divide scaling variant using BEHZ FullRNS techniques :  [1, 1, 1, 1, 0, 0, 1, 0]



[[1, 1, 976, 0, 1, 0, 0, 976], [1, 1, 1008, 0, 1, 0, 0, 1008]]



sk_power :  [[[1, 1, 976, 0, 1, 0, 0, 976], [1, 1, 1008, 0, 1, 0, 0, 1008]]]



phase [c0 + c1 * sk + c2 * sk^2 ...]_q. :  [[975, 120, 968, 4, 121, 106, 5, 969], [1007, 124, 1000, 4, 125, 110, 5, 1001]]



temp_product_modq :  [[975, 120, 968, 4, 121, 106, 5, 969], [1007, 124, 1000, 4, 125, 110, 5, 1001]]



result "Divide scaling variant using BEHZ FullRNS techniques :  [0, 1, 0, 0, 1, 1, 0, 0]



[[1, 1, 976, 0, 1, 0, 0, 976], [1, 1, 1008, 0, 1, 0, 0, 1008]]



sk

In [12]:
print(int_encoder.decode(dec1), "   ", int_encoder.decode(dec2), "   ", int_encoder.decode(dec3))

79     50     2


## Evaluator

In [13]:
eval = Evaluator(ctx)

In [14]:
cc12 = eval.add(ct1, ct2)
cc12 = decrypter.decrypt(cc12)
print(int_encoder.decode(cc12))

[[1, 1, 976, 0, 1, 0, 0, 976], [1, 1, 1008, 0, 1, 0, 0, 1008]]



sk_power :  [[[1, 1, 976, 0, 1, 0, 0, 976], [1, 1, 1008, 0, 1, 0, 0, 1008]]]



phase [c0 + c1 * sk + c2 * sk^2 ...]_q. :  [[120, 232, 117, 128, 130, 104, 131, 976], [124, 240, 121, 132, 134, 108, 135, 1008]]



temp_product_modq :  [[120, 232, 117, 128, 130, 104, 131, 976], [124, 240, 121, 132, 134, 108, 135, 1008]]



result "Divide scaling variant using BEHZ FullRNS techniques :  [1, 2, 1, 1, 1, 1, 1, 0]



129


In [15]:
pc12 = eval.add(pt1, ct2)
pc12 = decrypter.decrypt(pc12)
print(int_encoder.decode(pc12))

[[1, 1, 976, 0, 1, 0, 0, 976], [1, 1, 1008, 0, 1, 0, 0, 1008]]



sk_power :  [[[1, 1, 976, 0, 1, 0, 0, 976], [1, 1, 1008, 0, 1, 0, 0, 1008]]]



phase [c0 + c1 * sk + c2 * sk^2 ...]_q. :  [[120, 242, 113, 126, 121, 106, 127, 969], [124, 250, 117, 130, 125, 110, 131, 1001]]



temp_product_modq :  [[120, 242, 113, 126, 121, 106, 127, 969], [124, 250, 117, 130, 125, 110, 131, 1001]]



result "Divide scaling variant using BEHZ FullRNS techniques :  [1, 2, 1, 1, 1, 1, 1, 0]



129


In [16]:
pp12 = eval.add(pt1, pt2)
print(int_encoder.decode(pp12))

129


### Verify result

In [17]:
assert int_encoder.decode(cc12) == int_encoder.decode(pc12) == int_encoder.decode(pp12) == ri1+ri2

In [18]:
result = eval._mul_cipher_cipher(ct1, ct2)

result = decrypter.decrypt(result)

result = int_encoder.decode(result)
print('sk : ', sk.data)

print('final result: ', result)

[[1, 1, 976, 0, 1, 0, 0, 976], [1, 1, 1008, 0, 1, 0, 0, 1008]]



sk_power :  [[[1, 1, 976, 0, 1, 0, 0, 976], [1, 1, 1008, 0, 1, 0, 0, 1008]], [[2, 0, 976, 0, 3, 2, 974, 975], [2, 0, 1008, 0, 3, 2, 1006, 1007]]]



phase [c0 + c1 * sk + c2 * sk^2 ...]_q. :  [[19, 15, 7, 0, 50, 53, 17, 970], [39, 23, 23, 9, 35, 51, 31, 1004]]



temp_product_modq :  [[19, 15, 7, 0, 50, 53, 17, 970], [39, 23, 23, 9, 35, 51, 31, 1004]]



result "Divide scaling variant using BEHZ FullRNS techniques :  [5, 2, 4, 6, 0, 0, 3, 0]



sk :  [[1, 1, 976, 0, 1, 0, 0, 976], [1, 1, 1008, 0, 1, 0, 0, 1008]]
final result:  161


In [19]:
print(ri1 * ri2, "    ", result)
assert ri1 * ri2 == result

3950      161


AssertionError: 

In [20]:
print(decrypter._get_sufficient_sk_power(2)[0])
print('\n')
print(decrypter._get_sufficient_sk_power(2)[1])

[[1, 1, 976, 0, 1, 0, 0, 976], [1, 1, 1008, 0, 1, 0, 0, 1008]]



sk_power :  [[[1, 1, 976, 0, 1, 0, 0, 976], [1, 1, 1008, 0, 1, 0, 0, 1008]], [[2, 0, 976, 0, 3, 2, 974, 975], [2, 0, 1008, 0, 3, 2, 1006, 1007]]]



[[1, 1, 976, 0, 1, 0, 0, 976], [1, 1, 1008, 0, 1, 0, 0, 1008]]


[[1, 1, 976, 0, 1, 0, 0, 976], [1, 1, 1008, 0, 1, 0, 0, 1008]]



sk_power :  [[[1, 1, 976, 0, 1, 0, 0, 976], [1, 1, 1008, 0, 1, 0, 0, 1008]], [[2, 0, 976, 0, 3, 2, 974, 975], [2, 0, 1008, 0, 3, 2, 1006, 1007]]]



[[2, 0, 976, 0, 3, 2, 974, 975], [2, 0, 1008, 0, 3, 2, 1006, 1007]]


In [21]:
#Testing
from syft.frameworks.torch.he.fv.util.operations import poly_mul_mod

class Test:
    def __init__(self):
        self._secret_key_array = [[[1,2,3]]]
        self._coeff_modulus = [5]
    def _get_sufficient_sk_power(self, max_power):
            """Generate an list of secret key polynomial raised to 1...max_power.

            Args:
                max_power: heighest power up to which we want to raise secretkey.

            Returns:
                A 2-dim list having secretkey powers.
            """

            if max_power == len(self._secret_key_array):
                return self._secret_key_array

            while len(self._secret_key_array) < max_power:
                sk_extra_power = [0] * len(self._coeff_modulus)
                for i in range(len(self._coeff_modulus)):
                    sk_extra_power[i] = poly_mul_mod(self._secret_key_array[-1][i], self._secret_key_array[0][i], self._coeff_modulus[i])
                self._secret_key_array.append(sk_extra_power)

            return self._secret_key_array

In [22]:
print(Test()._get_sufficient_sk_power(2))

[[[1, 2, 3]], [[4, 0, 0]]]
